In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from scipy import stats
import requests
from requests import get
from bs4 import BeautifulSoup
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from linearmodels import PooledOLS
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [55]:
df_tr = pd.read_csv('transaction.csv', header = None, index_col = 0)
df_tr.columns = ['type', 'from', 'to', 'date', 'punk_id', 'amount_in_eth', 'amount_in_dol']
df_tr = df_tr[ ['punk_id'] + [ col for col in df_tr.columns if col != 'punk_id' ] ]
df_tr.head()

,punk_id,type,from,to,date,amount_in_eth,amount_in_dol
0,,,,,,,
0,0,Bid Withdrawn,0x983ace,NaN,2021-05-10,250.0,1030000.0
1,0,Bid,0x983ace,NaN,2021-04-11,250.0,"537,615"
2,0,Bid Withdrawn,0xd7510a,NaN,2021-03-03,100.0,"160,973"
3,0,Bid,0xd7510a,NaN,2021-02-20,100.0,"188,897"
4,0,Bid Withdrawn,natealex,NaN,2020-09-16,69.0,"25,532"


In [56]:
def fix_df(x):
    x['punk_id'] = x['punk_id'].astype('category')
    x['type'] = x['type'].astype('category')
    x['from'] = x['from'].astype(str)
    x['to'] = x['to'].astype(str)
    x['date'] = pd.to_datetime(x['date'], format = '%Y.%m.%d')
    
    x['amount_in_eth'] = x['amount_in_eth'].astype(str)
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Y', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Z', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('M', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('T', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('B', '')
    x['amount_in_eth'] = x['amount_in_eth'].replace(',', '', regex=True)
    x['amount_in_eth'] = x['amount_in_eth'].astype(float)

    x['amount_in_dol'] = x['amount_in_dol'].astype(str)
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Y', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('B', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('T', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('P', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Z', '')
    x['amount_in_dol'] = x['amount_in_dol'].replace('<', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].replace(',', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].astype(float)
    
    x['amount_in_eth'] = x['amount_in_eth'].fillna(0)
    x['amount_in_dol'] = x['amount_in_dol'].fillna(0)

    return x

In [57]:
df_tr = fix_df(df_tr)
df_tr.head()

,punk_id,type,from,to,date,amount_in_eth,amount_in_dol
0,,,,,,,
0,0,Bid Withdrawn,0x983ace,nan,2021-05-10,250.0,1030000.0
1,0,Bid,0x983ace,nan,2021-04-11,250.0,537615.0
2,0,Bid Withdrawn,0xd7510a,nan,2021-03-03,100.0,160973.0
3,0,Bid,0xd7510a,nan,2021-02-20,100.0,188897.0
4,0,Bid Withdrawn,natealex,nan,2020-09-16,69.0,25532.0


In [58]:
sales = df_tr[df_tr['type'].str.contains("Sold")].reset_index(drop=True)
sales = sales.drop(['from', 'to', 'type', 'amount_in_eth'], axis=1)

In [59]:
df_acc = pd.read_csv('accessories.csv', header=None, sep='\n')
df_acc = df_acc[0].str.split(',', expand=True)
df_acc.drop(0, axis=1, inplace=True)

dict={1:'gender', 
      2:  'acc_1',
      3:'acc_2', 
      4: 'acc_3',
      5:'acc_4',
      6: 'acc_5',
      7:'acc_6',
      8:'acc_7'}
df_acc.rename(columns=dict,
          inplace=True)
  
df_acc.head()

,gender,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7
0,Female,Green Eye Shadow,Earring,Blonde Bob,None,None,None,None
1,Male,Smile,Mohawk,None,None,None,None,None
2,Female,Wild Hair,None,None,None,None,None,None
3,Male,Wild Hair,Nerd Glasses,Pipe,None,None,None,None
4,Male,Big Shades,Wild Hair,Earring,Goat,None,None,None


In [60]:
types = df_acc['gender']
types_dummies = pd.get_dummies(types)
types_dummies['punk_id'] = types_dummies.index
types_dummies = types_dummies[ ['punk_id'] + [ col for col in types_dummies.columns if col != 'punk_id' ] ]

In [61]:
acc = pd.read_csv('accessories_transformed_full.csv', index_col = 0)
acc['punk_id'] = acc.index
acc = acc[ ['punk_id'] + [ col for col in acc.columns if col != 'punk_id' ] ]

In [62]:
df = pd.merge(sales, types_dummies, on="punk_id", how='left')
df = pd.merge(df, acc, on="punk_id", how='left')
df = df.set_index(['punk_id', 'date'])
df = df.fillna(0)
df

amount_in_dol  Alien  Ape  Female  Male  Zombie  Beanie  \
punk_id date                                                                  
0       2018-11-30         2822.0      0    0       1     0       0     0.0   
        2017-07-07          386.0      0    0       1     0       0     0.0   
        2017-06-23          320.0      0    0       1     0       0     0.0   
1       2020-11-30        36305.0      0    0       0     1       0     0.0   
        2019-04-06         5155.0      0    0       0     1       0     0.0   
...                           ...    ...  ...     ...   ...     ...     ...   
9990    2021-03-20        44981.0      0    0       0     1       0     0.0   
9997    2021-02-08       169770.0      0    0       0     0       1     0.0   
        2020-09-27        21062.0      0    0       0     0       1     0.0   
9998    2021-03-03        61170.0      0    0       1     0       0     0.0   
        2020-12-27         9499.0      0    0       1     0       0     0.0   

                    Choker  Pilot Helmet  Tiara  ...  Regular Shades  \
punk_id date                                     ...                   
0       2018-11-30     0.0           0.0    0.0  ...             0.0   
        2017-07-07     0.0           0.0    0.0  ...             0.0   
        2017-06-23     0.0           0.0    0.0  ...             0.0   
1       2020-11-30     0.0           0.0    0.0  ...             0.0   
        2019-04-06     0.0           0.0    0.0  ...             0.0   
...                    ...           ...    ...  ...             ...   
9990    2021-03-20     0.0           0.0    0.0  ...             0.0   
9997    2021-02-08     0.0           0.0    0.0  ...             0.0   
        2020-09-27     0.0           0.0    0.0  ...             0.0   
9998    2021-03-03     0.0           0.0    0.0  ...             0.0   
        2020-12-27     0.0           0.0    0.0  ...             0.0   

                    Horned Rim Glasses  Big Shades  Nerd Glasses  \
punk_id date                                                       
0       2018-11-30                 0.0         0.0           0.0   
        2017-07-07                 0.0         0.0           0.0   
        2017-06-23                 0.0         0.0           0.0   
1       2020-11-30                 0.0         0.0           0.0   
        2019-04-06                 0.0         0.0           0.0   
...                                ...         ...           ...   
9990    2021-03-20                 1.0         0.0           0.0   
9997    2021-02-08                 0.0         0.0           0.0   
        2020-09-27                 0.0         0.0           0.0   
9998    2021-03-03                 0.0         0.0           0.0   
        2020-12-27                 0.0         0.0           0.0   

                    Black Lipstick  Mole  Purple Lipstick  Hot Lipstick  \
punk_id date                                                              
0       2018-11-30             0.0   0.0              0.0           0.0   
        2017-07-07             0.0   0.0              0.0           0.0   
        2017-06-23             0.0   0.0              0.0           0.0   
1       2020-11-30             0.0   0.0              0.0           0.0   
        2019-04-06             0.0   0.0              0.0           0.0   
...                            ...   ...              ...           ...   
9990    2021-03-20             0.0   0.0              0.0           0.0   
9997    2021-02-08             0.0   0.0              0.0           0.0   
        2020-09-27             0.0   0.0              0.0           0.0   
9998    2021-03-03             1.0   0.0              0.0           0.0   
        2020-12-27             1.0   0.0              0.0           0.0   

                    Cigarette  Earring  
punk_id date                            
0       2018-11-30        0.0      1.0  
        2017-07-07        0.0      1.0  
        2017-06-23        0.0 

In [41]:
y = df['amount_in_dol']
X = df.iloc[:,1:]
X = X.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [44]:
exog = sm.tools.tools.add_constant(X_train.loc[:, X_train.columns != 'Male']) #To get a full rank matrix
exog = exog.astype(int)
endog = y_train
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:          amount_in_dol   R-squared:                        0.1723
Estimator:              RandomEffects   R-squared (Between):              0.1531
No. Observations:                9471   R-squared (Within):               0.0000
Date:                Thu, Jun 24 2021   R-squared (Overall):              0.1723
Time:                        17:58:12   Log-likelihood                -1.211e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      21.449
Entities:                        5591   P-value                           0.0000
Avg Obs:                       1.6940   Distribution:                 F(91,9379)
Min Obs:                       0.0000                                           
Max Obs:                       13.000   F-statistic (robust):             21.449
                            

In [188]:
y_pred = re_res.predict(X_test)
y_pred

,,predictions
punk_id,date,
3826,2020-09-24,20517.769356
1163,2021-05-03,223867.220361
4271,2021-04-08,18606.461575
1719,2021-03-12,33341.168247
9417,2017-11-26,227985.175489
...,...,...
9749,2021-02-08,41823.845059
9155,2020-08-04,17737.813645
5561,2018-01-23,28295.520854


In [190]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 76214.79791353409
Mean Squared Error: 30269493994.913723
Root Mean Squared Error: 173981.30357861365


In [51]:
model2  = PanelOLS(endog, exog, time_effects=True)
pols = model2.fit()
print(pols)

                          PanelOLS Estimation Summary                           
Dep. Variable:          amount_in_dol   R-squared:                        0.2180
Estimator:                   PanelOLS   R-squared (Between):              0.1201
No. Observations:                9471   R-squared (Within):               0.0000
Date:                Thu, Jun 24 2021   R-squared (Overall):              0.1658
Time:                        18:02:20   Log-likelihood                -1.202e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      25.845
Entities:                        5591   P-value                           0.0000
Avg Obs:                       1.6940   Distribution:                 F(91,8437)
Min Obs:                       0.0000                                           
Max Obs:                       13.000   F-statistic (robust):             25.845
                            

In [52]:
y_pred2 = pols.predict(X_test)
y_pred2

,,predictions
punk_id,date,
3826,2020-09-24,-25.048708
1163,2021-05-03,263732.333192
4271,2021-04-08,263.474083
1719,2021-03-12,13843.635155
9417,2017-11-26,289903.991144
...,...,...
9749,2021-02-08,50941.257727
9155,2020-08-04,1378.243522
5561,2018-01-23,12334.525431


In [53]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

Mean Absolute Error: 91353.20387370537
Mean Squared Error: 36416954313.65309
Root Mean Squared Error: 190832.26748548867
